In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.5 MB/s eta 0:00:00


In [2]:
!pip install feature_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install plot-metric

In [4]:
!pip install scipy==1.11.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [5]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from plot_metric.functions import BinaryClassification

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [6]:
import json
data = {
  "account": "",
  "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
  "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
  "quota_project_id": "dev-sd-lake",
  "refresh_token": "1//05nz9IRnR1e2JCgYIARAAGAUSNwF-L9Ir_zi7EkfNFJb02e5XxL3e4Qicf7BN708CjMo66DuPPMhkd09jS6lIra-zubUPFGORcyU",
  "type": "authorized_user",
  "universe_domain": "googleapis.com"
}
json_data = json.dumps(data)
!echo '{json_data}' > '/content/.config/application_default_credentials.json'

In [7]:
from google.cloud import bigquery

client = bigquery.Client(project='dev-sd-lake')

product_listings_query = """
SELECT * FROM pollen.product_listings;
"""

product_listings = client.query_and_wait(product_listings_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

products_query = """
SELECT * FROM pollen.products;
"""

products = client.query_and_wait(products_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_categories_query = """
SELECT * FROM pollen.product_categories;
"""

product_categories = client.query_and_wait(product_categories_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_subcategories_query = """
SELECT * FROM pollen.product_subcategories;
"""

product_subcategories = client.query_and_wait(product_subcategories_query).to_dataframe()

client = bigquery.Client(project='dev-sd-lake')

sellers_query = """
SELECT * FROM pollen.sellers;
"""

sellers = client.query_and_wait(sellers_query).to_dataframe()

In [8]:
offers_query = """
SELECT * FROM pollen.offers;
"""

offers = client.query_and_wait(offers_query).to_dataframe()

In [9]:
orders_level_1_query = """
SELECT * FROM pollen.orders_level_1;
"""

orders_level_1 = client.query_and_wait(orders_level_1_query).to_dataframe()

In [10]:
orders_level_2_query = """
SELECT * FROM pollen.orders_level_2;
"""

orders_level_2 = client.query_and_wait(orders_level_2_query).to_dataframe()

In [11]:
sellers = sellers[['seller_name', 'persona_seller_type']].drop_duplicates()

In [12]:
print(product_listings.shape, products.shape, product_categories.shape, product_subcategories.shape, sellers.shape)

(99164, 41) (99172, 9) (22, 3) (111, 4) (109, 2)


In [13]:
def get_order_date(df_order_sheet1, df_order_sheet2):
  order_date_to_tracking_no_mapping = df_order_sheet1[['tracking_no', 'order_date']].drop_duplicates()
  df_order_sheet2 = pd.merge(df_order_sheet2, order_date_to_tracking_no_mapping, 'left',  on = ['tracking_no'])
  df_order_sheet2.drop('date_of_order', axis = 1, inplace = True)
  df_order = df_order_sheet2.copy()
  return df_order

orders = get_order_date(orders_level_1, orders_level_2)

In [14]:
orders = orders.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.rename(columns = {'total_units' : 'total_units_offered'})
orders = orders.rename(columns = {'total_units' : 'total_units_ordered', 'total_offer_price_usd' : 'total_order_price_usd'})

offers = pd.merge(offers, orders[['sku_number', 'expiry_date', 'tracking_no', 'total_units_ordered', 'recovery_rate_percentage', 'total_order_price_usd']], 'left', on = ['sku_number', 'expiry_date', 'tracking_no'])

In [15]:
product_subcategories = product_subcategories.rename(columns = {'id' : 'subcategory_id', 'name' : 'product_subcategory'})
product_categories = product_categories.rename(columns = {'id' : 'category_id', 'name' : 'product_category'})

products = pd.merge(products, product_categories, 'left', on = 'category_id')
products = pd.merge(products, product_subcategories[['subcategory_id', 'product_subcategory']], 'left', on = 'subcategory_id')

In [16]:
products = products.rename(columns = {'id' : 'product_id'})
products = products.drop(['parent_product_id', 'created_at', 'updated_at', 'category_id', 'subcategory_id'], axis = 1)

product_listings = product_listings.drop(['created_at', 'updated_at', 'pollen_updated_price_per_unit_local',
                       'barcode', 'barcode_key', 'manufactured_date', 'batch_number', 'image_links', 'usd_conversion', 'scoring'], axis = 1)
product_listings = pd.merge(product_listings, products, 'left', on = 'product_id')

In [17]:
product_listings.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,7c765b8d-4858-42f3-bdcd-d39e07146f58,7468eef6-5131-48d9-89d3-e13006c70311,SE1712631761502,SE1712631761502G4311200,L'Oreal Indonesia,2025-01-01,Carton,64.20,60.0,3852.0,IDR,24900.0,5991.5,31.0,g,0.01,254.0,156.0,219.0,MM,2.24,Bekasi,Indonesia,-,-,NaN,P2,1.53,0.37,CORE,2024-05-28,G4311200,GRN 2 STEP PW MASK B5 GOJI EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS
1,b419d6ba-dd0c-4283-83de-df3a38e7a9be,9eb95253-892b-4eea-9fa0-33a79786d3d1,SE1712631761502,SE1712631761502G4218000,L'Oreal Indonesia,2025-01-28,Carton,58.43,60.0,3506.0,IDR,23400.0,6552.7,19.7,g,0.01,185.0,150.0,197.0,MM,1.42,Bekasi,Indonesia,-,-,NaN,P2,1.44,0.40,CORE,2024-05-28,G4218000,GRN 5 MIN AMP MASK WATERMELON EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS


In [18]:
product_listings.inventory_class.value_counts()

,count
inventory_class,
CORE,66703
NON_CORE,32461


In [19]:
df_inv = product_listings[product_listings.inventory_class == 'CORE']
non_core_inv = product_listings[product_listings.inventory_class == 'NON_CORE']
df_inv = df_inv.reset_index(drop = True)
non_core_inv = non_core_inv.reset_index(drop = True)

In [20]:
cols_list_underscore_cleaning = ['sku_product_name', 'brand', 'product_category', 'product_subcategory', 'warehouse_country',
            'country_of_origin', 'pack_label_language', 'product_restricted_countries', 'shelf_life_bucket']

inv_cols_list_to_change_dtypes = ['qty_of_cartons', 'units_per_cartons', 'retail_price_per_case_(local)', 'retail_price_per_case_(usd)', 'asking_price_per_case_(local)',
                              'asking_price_per_case_(usd)', 'total_retail_price_(local)', 'total_retail_price_(usd)', 'total_asking_price_(local)',
                              'total_asking_price_(usd)', 'pack_size_(number)', 'total_cbm', 'cbm_per_case', 'package_dimensions_per_package_type_(length)',
                              'package_dimensions_per_case_(width)', 'package_dimensions_per_case_(height)', 'net_weight_\nper_unit(kg)', 'net_weight_per_case_(kg)',
                              'gross_weight_per_case_(kg)', 'total_net_weight_(kg)', 'total_gross_weight_(kg)', 'cases_per_pallet', 'number_of_pallets', 'discount',
                              'shelf_remaining_days']

record_attributes = ['sku_number', 'sku_name', 'brand', 'product_category', 'product_subcategory', 'retail_price_per_unit_local', 'currency']

n_days_before_date = pd.to_datetime(date.today() - timedelta(days = 14))

THRESHOLD = 0.65

In [21]:
def dropping_null_brand_cat_subcats(df_inv):
  df_inv = df_inv[~df_inv.brand.isnull()]
  df_inv = df_inv[~df_inv.product_category.isnull()]
  df_inv = df_inv[~df_inv.product_subcategory.isnull()]
  return df_inv

def cols_to_lower_rem_space(df, cols_list):
  for col in ['sku_name', 'product_name', 'brand', 'brand_', 'product_description', 'product_category', 'product_sub_category', 'product_subcategory',\
              'warehouse_location', 'buyer', 'seller', 'package_type', 'deal_type', 'origin_', 'destination', 'region_of_export', \
              'domestic_export', 'order_type', 'country', 'currency', 'country_of_origin', 'pack_label_language', 'dangerous_goods\n(y/n)', 'shelf_life_bucket',
              'relavant_(r)/ir-relavant(ir)', 'product_restricted_countries', 'store_name', 'quarter', 'temp_reference', 'tracking_no', 'offer_type', 'fiscal_month',
              'proposed_buyer_product_preferences', 'product_shelf_life_months', 'priority', 'seller', 'lms_seller_id', 'persona', 'core_vs._\nnon_core', 'sku_product_name',
              'warehouse_country', 'warehouse_address', 'measurement_units', 'listing_currency', 'manufacturing_country', 'lbh_measurement_units', 'package_type', 'seller_name',
              'pack_size_unit', 'package_labeled_language', 'inventory_class', 'description', 'unit_dimensions']:
              if col in cols_list:
                df[col] = df[col].str.lower().str.replace(' ', '_')
  return df

def remove_starting_and_trailing_underscores(x):
  if x != '':
    if x[0] == '_':
      for i in range(len(x)):
        if x[i] != '_':
          start = i
          break
      x = x[start:]
    if x[-1] == '_':
      for i in reversed(range(len(x))):
        if x[i] != '_':
          end = i
          break
      x = x[:end+1]
  return x

def cleaning_underscores_inv_data(df_inv, col_list):
  for col in col_list:
    print(col)
    df_inv[col] = df_inv[col].apply(remove_starting_and_trailing_underscores)
  return df_inv

def drop_dups_and_clean(df_inv):
  print('Dropping above duplicates ...')
  df_inv = df_inv.drop_duplicates()
  return df_inv

In [22]:
df_inv = dropping_null_brand_cat_subcats(df_inv)
df_inv = cols_to_lower_rem_space(df_inv, df_inv.columns)
df_inv = drop_dups_and_clean(df_inv)

non_core_inv = cols_to_lower_rem_space(non_core_inv, non_core_inv.columns)

Dropping above duplicates ...


# Non Core

In [23]:
non_core_inv.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,2d28d351-f531-42d7-abc1-fee72e225c53,1e18e385-f133-4fb3-95f4-b56cb1419369,SE1712631761542,SE171263176154204R3I1MKJPD97H7OBZ60UTU1Z,acommerce_singapore,2023-05-31,pieces,NaN,NaN,6.0,sgd,19.9,1.99,NaN,None,0.0,10.0,10.0,10.0,cm,0.50,sg-acom01,None,None,None,NaN,p1,15.116504,1.511650,non_core,2023-01-25,04R3I1MKJPD97H7OBZ60UTU1Z,l'oreal_paris_color_riche_satin_105_nude_in_paris,l'oreal_paris,makeup,makeup,lip_makeup
1,063c43ca-8297-4bea-9bbf-3ff2e9fd8376,3a7331cf-dbd4-4cb4-b791-982833ff97ff,SE1712631761542,SE171263176154211YAP8LR1W3OQSFJGWCLTD1DQ,acommerce_singapore,2023-05-31,pieces,NaN,NaN,160.0,sgd,2.0,0.20,NaN,None,0.0,8.0,5.0,0.2,cm,0.15,sg-acom02,None,None,None,NaN,p1,1.519247,0.151925,non_core,2023-01-25,11YAP8LR1W3OQSFJGWCLTD1DQ,garnier_serum_mask_-_pomegranate_(replumping),garnier,skin_care,skin_care,masks_&_exfoliators


In [24]:
non_core_inv = non_core_inv.rename(columns = {'total_number_of_items' : 'total_units'})
non_core_inv['updated_on'] = pd.to_datetime(non_core_inv['updated_on'])
non_core_inv['expiry_date'] = pd.to_datetime(non_core_inv['expiry_date'])

In [25]:
print(non_core_inv.shape)
non_core_inv = non_core_inv.drop_duplicates(subset = ['sku_number', 'expiry_date', 'updated_on', 'seller_name'])
print(non_core_inv.shape)

(32461, 37)
(29476, 37)


In [26]:
offers = offers.rename(columns = {'seller' : 'seller_name'})
offers['date_of_offer'] = pd.to_datetime(offers['date_of_offer'])
offers['seller_name'] = offers['seller_name'].str.lower()
offers['total_units_ordered'] = offers['total_units_ordered'].str.replace(',', '')
offers['total_units_ordered'] = offers['total_units_ordered'].astype(float)

In [27]:
def merge_non_core_inv_and_offer(non_core_inv, df_offer):
  temp = pd.merge(non_core_inv, df_offer[['sku_number', 'seller_name', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                          'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd']], 'left', on = ['sku_number', 'seller_name'])
  # temp['updated_on'] = pd.to_datetime(temp['updated_on'])
  temp['offer_day_diff_updated_inv'] = (temp['date_of_offer'] - temp['updated_on']).dt.days
  # avoiding future entry from same sku and seller to get filtered out hence nulling it as will remain in the dataframe
  temp['offer_day_diff_updated_inv'] = np.where(temp.offer_day_diff_updated_inv < 0, np.nan, temp['offer_day_diff_updated_inv'])
  temp = temp[(temp.offer_day_diff_updated_inv >= 0) | (temp.offer_day_diff_updated_inv.isnull())]
  df_non_core_inv_min_offer_day_diff_updated_inv = temp.groupby(['sku_number', 'seller_name', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                        .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  temp = pd.merge(temp, df_non_core_inv_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'seller_name', 'date_of_offer'])
  temp = temp[(temp.offer_day_diff_updated_inv == temp.min_offer_day_diff_updated_inv) | (temp.offer_day_diff_updated_inv.isnull())]
  temp.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return temp

In [28]:
non_core_inv = merge_non_core_inv_and_offer(non_core_inv, offers)
# non_core_inv['sell_thru_rate'] = non_core_inv['total_units_ordered'] / non_core_inv['total_units']
# non_core_inv['sell_thru_rate'] = np.where(non_core_inv['sell_thru_rate'] > 1, 1, non_core_inv['sell_thru_rate'])

In [29]:
non_core_inv['shelf_life_remaining_days'] = (non_core_inv['expiry_date'] - non_core_inv['updated_on']).dt.days

In [30]:
temp = non_core_inv[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units', 'shelf_life_remaining_days',
                     'warehouse_country', 'total_units_ordered', 'recovery_rate_percentage']].sort_values(by = ['sku_number', 'brand', 'product_category', 'product_subcategory',
                      'expiry_date', 'seller_name', 'updated_on', 'priority', 'warehouse_country'])
temp = temp.reset_index(drop = True)

In [31]:
def get_prev_inv_occurence_next_to_current_for_non_core_inv(temp):
  new_col_names_for_shifted_inv = dict()
  for i in temp.columns:
    new_col_names_for_shifted_inv[i] = i+'_prev'

  temp_shifted = temp.shift(1)
  temp_shifted = temp_shifted.rename(columns = new_col_names_for_shifted_inv)
  temp = pd.concat([temp, temp_shifted], axis=1)
  return temp

# not taking expiry into account

def get_repeated_inv_and_mean_depletion_for_non_core_inv(temp):
  repeated_inv = temp[(temp.sku_number == temp.sku_number_prev) & (temp.seller_name == temp.seller_name_prev)]
  repeated_inv['depletion'] = repeated_inv['total_units_prev'] - repeated_inv['total_units']
  repeated_inv['time'] = (repeated_inv['updated_on'] - repeated_inv['updated_on_prev']).dt.days
  repeated_inv = repeated_inv.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', \
                            'seller_name', 'updated_on', 'priority'])[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on',\
                                                                        'priority', 'depletion', 'time', 'total_units_prev', 'total_units', 'priority_prev',\
                                                                        'shelf_life_remaining_days_prev', 'warehouse_country_prev', 'total_units_ordered_prev',
                                                                        'recovery_rate_percentage_prev']]

  return repeated_inv

## Recommendation - measure depletion at some other level, exclude skus as many skus don't repeat and many rows are getting dropped

In [32]:
temp = get_prev_inv_occurence_next_to_current_for_non_core_inv(temp)
repeated_inv_non_core = get_repeated_inv_and_mean_depletion_for_non_core_inv(temp)
repeated_inv_non_core = repeated_inv_non_core.reset_index(drop = True)
repeated_inv_non_core['priority'] = np.where(repeated_inv_non_core.priority != repeated_inv_non_core.priority_prev, \
                                                   repeated_inv_non_core.priority_prev, repeated_inv_non_core.priority)

In [33]:
# 90 days sales life cycle
repeated_inv_non_core = repeated_inv_non_core[repeated_inv_non_core.time <= 90]

mean_depletion_per_cat_non_core_inv = repeated_inv_non_core.groupby(['product_category']).depletion.mean().reset_index().sort_values(by = 'depletion')

low_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(0.33)].product_category)
mid_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(0.67)].product_category)
mid_depletion_cats = [i for i in mid_depletion_cats if i not in low_depletion_cats]
high_depletion_cats = list(mean_depletion_per_cat_non_core_inv[mean_depletion_per_cat_non_core_inv['depletion'] <= mean_depletion_per_cat_non_core_inv.depletion.quantile(1)].product_category)
high_depletion_cats = [i for i in high_depletion_cats if i not in low_depletion_cats + mid_depletion_cats]

print(low_depletion_cats, '\n', mid_depletion_cats, '\n', high_depletion_cats)

['beverages', 'health_&_supplments', 'industrial', 'oral_care', 'skin_care'] 
 ['baby', 'hair_care', 'home_care', 'fashion_-_accessories'] 
 ['fragrance', 'sports,_fitness,_&_outdoors', 'makeup', 'home', 'food']


In [34]:
def set_depletion_non_core(row):
  if row['depletion'] < 0:
    return row['total_units_prev']
  elif row['product_category'] in high_depletion_cats and row['time'] <= 20:
    return row['depletion']
  elif row['product_category'] in high_depletion_cats and row['time'] > 20:
    return row['total_units_prev']
  elif row['product_category'] in mid_depletion_cats and row['time'] <= 45:
    return row['depletion']
  elif row['product_category'] in mid_depletion_cats and row['time'] > 45:
    return row['total_units_prev']
  elif row['product_category'] in low_depletion_cats and row['time'] <= 60:
    return row['depletion']
  elif row['product_category'] in low_depletion_cats and row['time'] > 60:
    return row['total_units_prev']

In [35]:
repeated_inv_non_core['depletion'] = repeated_inv_non_core.apply(set_depletion_non_core, axis=1)

In [36]:
def remove_percent_sign(x):
  if pd.isna(x):
    return x
  else:
    return x[:-1]
repeated_inv_non_core['recovery_rate_percentage_prev'] = repeated_inv_non_core['recovery_rate_percentage_prev'].apply(remove_percent_sign)
repeated_inv_non_core['recovery_rate_percentage_prev'] = np.where(repeated_inv_non_core['recovery_rate_percentage_prev'] == '', np.nan, repeated_inv_non_core['recovery_rate_percentage_prev'])
repeated_inv_non_core['recovery_rate_percentage_prev'] = repeated_inv_non_core['recovery_rate_percentage_prev'].astype(float)
repeated_inv_non_core['recovery_rate_percentage_prev'] = np.where(repeated_inv_non_core['recovery_rate_percentage_prev'].isnull(), 0 , repeated_inv_non_core['recovery_rate_percentage_prev'])
repeated_inv_non_core['total_units_ordered_prev'] = np.where(repeated_inv_non_core['total_units_ordered_prev'].isnull(), 0, repeated_inv_non_core['total_units_ordered_prev'])

In [37]:
repeated_inv_non_core = repeated_inv_non_core.rename(columns = {'total_units_ordered_prev' : 'pollen_liquidated_units',
                                                                'recovery_rate_percentage_prev' : 'pollen_recovery_rate_percentage'})

In [38]:
repeated_inv_non_core.sample(2)

,sku_number,brand,product_category,product_subcategory,expiry_date,seller_name,updated_on,priority,depletion,time,total_units_prev,total_units,priority_prev,shelf_life_remaining_days_prev,warehouse_country_prev,pollen_liquidated_units,pollen_recovery_rate_percentage
179,3CEID0052,3_ce,makeup,lip_makeup,2024-07-14,eblo,2023-10-25,p1,170.0,72,170.0,156.0,p1,335.0,None,0.0,0.0
348,3CEID012570,3_ce,makeup,face_makeup,2025-08-22,eblo,2024-06-13,p1,102.0,56,102.0,69.0,p1,491.0,None,0.0,0.0


# Core

In [39]:
offers = offers.rename(columns = {'seller_name' : 'seller_short'})

seller_name_to_short_form = {'unilever_indonesia' : 'ulid', 'unilever_malaysia' : 'ulmy', 'unilever_philippines' : 'ulph', 'unilever_thailand' : 'ulth', 'unilever_singapore' : 'ulsg',
                             "l'oreal_malaysia" : "lomy", "l'oreal_philippines" : 'loph', "l'oreal_thailand" : "loth", "l'oreal_india" : "loin", "l'oreal_indonesia" : "loid"}

df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])

def apply_seller_short(x):
  if x in seller_name_to_short_form.keys():
    return seller_name_to_short_form[x]
  else:
    return x

df_inv['seller_short'] = df_inv['seller_name'].apply(apply_seller_short)

offers = offers[offers['expiry_date'] != '-']
offers['expiry_date'] = pd.to_datetime(offers['expiry_date'], format = "%d-%b-%Y")
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])

In [40]:
def merge_inv_and_offer(df_inv, df_offer):
  df_inv = pd.merge(df_inv, df_offer[['sku_number', 'expiry_date', 'seller_short', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                      'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd']], 'left', on = ['sku_number', 'expiry_date', 'seller_short'])
  df_inv['offer_day_diff_updated_inv'] = (df_inv['date_of_offer'] - df_inv['updated_on']).dt.days
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv >= 0) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_min_offer_day_diff_updated_inv = df_inv.groupby(['sku_number', 'expiry_date', 'seller_short', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                      .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  df_inv = pd.merge(df_inv, df_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'expiry_date', 'seller_short', 'date_of_offer'])
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv == df_inv.min_offer_day_diff_updated_inv) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_inv.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return df_inv

In [41]:
df_inv = merge_inv_and_offer(df_inv, offers)
df_inv['total_units_ordered'] = df_inv['total_units_ordered'].fillna(0)
# df_inv['sell_thru_rate'] = df_inv['total_units_ordered'] / df_inv['total_number_of_items']
# df_inv['sell_thru_rate'] = np.where(df_inv['sell_thru_rate'] > 1, 1, df_inv['sell_thru_rate'])
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

In [42]:
# dropping duplicates considering all columns except the ids
k = list(set(df_inv.columns) - set([i for i in df_inv.columns if i.endswith('id')]))
df_inv = df_inv.drop_duplicates(subset = k)

In [43]:
df_inv = df_inv.rename(columns = {'total_number_of_items' : 'total_units'})
df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])
df_inv = df_inv[df_inv.sku_number != '']
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

def remove_percent_sign(x):
  if pd.isna(x):
    return x
  else:
    return x[:-1]
df_inv['recovery_rate_percentage'] = df_inv['recovery_rate_percentage'].apply(remove_percent_sign)
df_inv['recovery_rate_percentage'] = np.where(df_inv['recovery_rate_percentage'] == '', np.nan, df_inv['recovery_rate_percentage'])
df_inv['recovery_rate_percentage'] = df_inv['recovery_rate_percentage'].astype(float)
df_inv['recovery_rate_percentage'] = np.where(df_inv['recovery_rate_percentage'].isnull(), 0 , df_inv['recovery_rate_percentage'])

In [44]:
key_to_total_units_ordered = df_inv.groupby(['sku_number', 'seller_name', 'expiry_date', 'updated_on']).agg({'total_units_ordered' : 'sum'}).reset_index().rename(columns =
                                                {'total_units_ordered' : 'total_units_ordered_sum'})
df_inv = pd.merge(df_inv, key_to_total_units_ordered, 'left', on = ['sku_number', 'seller_name', 'expiry_date', 'updated_on'])

In [45]:
temp = df_inv[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units',
               'shelf_life_remaining_days', 'warehouse_country', 'total_units_ordered_sum', 'recovery_rate_percentage']]\
                  .sort_values(by = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'warehouse_country'])
temp = temp.reset_index(drop = True)

In [46]:
print(temp.shape)
print(temp.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on', 'priority', 'total_units']).shape)

(54892, 13)
(51240, 13)


In [47]:
def get_prev_inv_occurence_next_to_current(temp):
  new_col_names_for_shifted_inv = dict()
  for i in temp.columns:
    new_col_names_for_shifted_inv[i] = i+'_prev'

  temp_shifted = temp.shift(1)
  temp_shifted = temp_shifted.rename(columns = new_col_names_for_shifted_inv)
  temp = pd.concat([temp, temp_shifted], axis=1)
  return temp

def get_repeated_inv_and_mean_depletion(temp):
  repeated_inv = temp[(temp.sku_number == temp.sku_number_prev) & (temp.expiry_date == temp.expiry_date_prev) & (temp.seller_name == temp.seller_name_prev)]
  repeated_inv['depletion'] = repeated_inv['total_units_prev'] - repeated_inv['total_units']
  repeated_inv['time'] = (repeated_inv['updated_on'] - repeated_inv['updated_on_prev']).dt.days
  repeated_inv = repeated_inv.drop_duplicates(subset = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', \
                            'seller_name', 'updated_on', 'priority'])[['sku_number', 'brand', 'product_category', 'product_subcategory', 'expiry_date', 'seller_name', 'updated_on',\
                                                                        'priority', 'depletion', 'time', 'total_units_prev', 'total_units', 'priority_prev',\
                                                                        'shelf_life_remaining_days_prev', 'warehouse_country_prev', 'total_units_ordered_sum_prev',
                                                                        'recovery_rate_percentage_prev']]
  return repeated_inv

In [48]:
temp = get_prev_inv_occurence_next_to_current(temp)
repeated_inv = get_repeated_inv_and_mean_depletion(temp)

# since current was match to previous, hence correcting the current row previous priority order to previous one
repeated_inv['priority'] = np.where(repeated_inv.priority != repeated_inv.priority_prev, repeated_inv.priority_prev, repeated_inv.priority)

In [49]:
print(repeated_inv[repeated_inv.time > 90].shape)
print('Removing above number of rows as sales cycle is fixed to 90 days max for any cat or sub cat after which it is assumed to finish')
repeated_inv = repeated_inv[repeated_inv.time <= 90]

(2899, 17)
Removing above number of rows as sales cycle is fixed to 90 days max for any cat or sub cat after which it is assumed to finish


In [50]:
mean_depletion_per_cat_core_inv = repeated_inv.groupby(['product_category']).depletion.mean().reset_index().sort_values(by = 'depletion')

In [51]:
low_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(0.33)].product_category)
mid_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(0.67)].product_category)
mid_depletion_cats = [i for i in mid_depletion_cats if i not in low_depletion_cats]
high_depletion_cats = list(mean_depletion_per_cat_core_inv[mean_depletion_per_cat_core_inv['depletion'] <= mean_depletion_per_cat_core_inv.depletion.quantile(1)].product_category)
high_depletion_cats = [i for i in high_depletion_cats if i not in low_depletion_cats + mid_depletion_cats]

In [52]:
print(low_depletion_cats, '\n', mid_depletion_cats, '\n', high_depletion_cats)

['beverages', 'fashion_-_accessories', 'home', 'oral_care'] 
 ['makeup', 'fragrance', 'skin_care'] 
 ['baby', 'hair_care', 'home_care', 'food']


In [53]:
# do disposal correction

In [54]:
def set_depletion(row):
  if row['depletion'] < 0:
    return row['total_units_prev']
  elif row['product_category'] in high_depletion_cats and row['time'] <= 20:
    return row['depletion']
  elif row['product_category'] in high_depletion_cats and row['time'] > 20:
    return row['total_units_prev']
  elif row['product_category'] in mid_depletion_cats and row['time'] <= 45:
    return row['depletion']
  elif row['product_category'] in mid_depletion_cats and row['time'] > 45:
    return row['total_units_prev']
  elif row['product_category'] in low_depletion_cats and row['time'] <= 60:
    return row['depletion']
  elif row['product_category'] in low_depletion_cats and row['time'] > 60:
    return row['total_units_prev']

In [55]:
repeated_inv['depletion'] = repeated_inv.apply(set_depletion, axis=1)
repeated_inv = repeated_inv.reset_index(drop = True)

In [56]:
repeated_inv = repeated_inv.rename(columns = {'total_units_ordered_sum_prev' : 'pollen_liquidated_units', 'recovery_rate_percentage_prev' : 'pollen_recovery_rate_percentage'})

In [57]:
COLUMNS_FOR_DEPLETION_MODELING = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'time']

In [58]:
repeated_inv = repeated_inv[COLUMNS_FOR_DEPLETION_MODELING + ['priority', 'total_units_prev', 'depletion', 'shelf_life_remaining_days_prev', 'warehouse_country_prev',
                                                              'pollen_liquidated_units', 'pollen_recovery_rate_percentage']]
repeated_inv_non_core = repeated_inv_non_core[COLUMNS_FOR_DEPLETION_MODELING + ['priority', 'total_units_prev', 'depletion', 'shelf_life_remaining_days_prev', 'warehouse_country_prev',
                                                                                'pollen_liquidated_units', 'pollen_recovery_rate_percentage']]
print(repeated_inv.shape, repeated_inv_non_core.shape)

(19109, 13) (979, 13)


In [59]:
repeated_inv = pd.concat([repeated_inv, repeated_inv_non_core], ignore_index=True)
repeated_inv.depletion['depletion'] = np.where(repeated_inv.depletion < repeated_inv.pollen_liquidated_units, repeated_inv.pollen_liquidated_units, repeated_inv.depletion)
repeated_inv.shape

(20088, 13)

In [61]:
def correct_p1_depletion(df):
  if df['priority'] == 'p1' and np.random.uniform(low=1, high=99, size=(1,))[0] <= 80:
    return np.random.uniform(low=5, high=20, size=(1,))[0]/ 100 * df['depletion']
  else:
    return df['depletion']

repeated_inv['depletion'] = repeated_inv.apply(correct_p1_depletion, axis = 1)
repeated_inv['pollen_liquidated_units'] = np.where(repeated_inv.pollen_liquidated_units > repeated_inv.depletion, repeated_inv.depletion, repeated_inv.pollen_liquidated_units)
repeated_inv = repeated_inv.reset_index(drop = True)
repeated_inv['possible'] = 1

In [70]:
low_shelf_life_dummy = repeated_inv.copy()

def get_dummy_shelf_life_not_possible(df):
    if pd.isna(df['shelf_life_remaining_days_prev']):
      return df['shelf_life_remaining_days_prev']
    else:
      return np.random.uniform(
            low=0.5 * df['shelf_life_remaining_days_prev'],
            high=df['shelf_life_remaining_days_prev'], size=(1,))[0]

low_shelf_life_dummy['shelf_life_remaining_days_prev'] = low_shelf_life_dummy.apply(get_dummy_shelf_life_not_possible, axis = 1)
low_shelf_life_dummy['possible'] = 0

In [71]:
low_time_dummy = repeated_inv.copy()

def get_dummy_time_not_possible(df):
    if pd.isna(df['time']):
      return df['time']
    else:
      return np.random.uniform(
            low=0.5 * df['time'],
            high=df['time'], size=(1,))[0]

low_time_dummy['time'] = low_time_dummy.apply(get_dummy_time_not_possible, axis = 1)
low_time_dummy['possible'] = 0

In [73]:
high_depletion_dummy = repeated_inv.copy()

def get_dummy_high_depletion(df):
    if pd.isna(df['depletion']):
      return df['depletion']
    else:
      return np.random.uniform(
            low=df['depletion'],
            high=1.5 * df['depletion'], size=(1,))[0]

high_depletion_dummy['depletion'] = high_depletion_dummy.apply(get_dummy_high_depletion, axis = 1)
high_depletion_dummy['total_units_prev'] = np.where(high_depletion_dummy['total_units_prev'] < high_depletion_dummy['depletion'],
                                                    high_depletion_dummy['depletion'], high_depletion_dummy['total_units_prev'])
high_depletion_dummy['possible'] = 0

In [75]:
repeated_inv = pd.concat([repeated_inv, low_shelf_life_dummy, low_time_dummy, high_depletion_dummy], ignore_index=True)
repeated_inv = repeated_inv.drop_duplicates()
repeated_inv = repeated_inv.reset_index(drop = True)

## Listing type

In [78]:
def get_listing_type(df):
  if df['shelf_life_remaining_days_prev'] > 365*2:
    return 'fresh'
  elif df['shelf_life_remaining_days_prev'] > 365 and df['pollen_recovery_rate_percentage'] > 0 and df['pollen_recovery_rate_percentage'] < 10:
    return 'obsolete'
  else:
    return 'excess'

repeated_inv['listing_condition'] = repeated_inv.apply(get_listing_type, axis = 1)

number_of_rows_to_randomize = int(repeated_inv.shape[0]*0.05)

change = repeated_inv.sample(number_of_rows_to_randomize//2).index
repeated_inv.loc[change,'listing_condition'] = 'damaged'
change = repeated_inv.sample(number_of_rows_to_randomize//2).index
repeated_inv.loc[change,'listing_condition'] = 'made_to_order'

In [79]:
repeated_inv.listing_condition.value_counts()

,count
listing_condition,
excess,71222
fresh,3105
made_to_order,1958
damaged,1910
obsolete,155


## Liquidation channel

In [80]:
repeated_inv['depletion'] = np.where(repeated_inv['depletion'] < repeated_inv['pollen_liquidated_units'], repeated_inv['pollen_liquidated_units'], repeated_inv['depletion'])
repeated_inv['depletion_other_than_pollen'] = repeated_inv['depletion'] - repeated_inv['pollen_liquidated_units']
repeated_inv['liquidation_channel'] = [['friends/family', 'distributor', 'retailer', 'liquidator', 'disposal', 'pollen']]*len(repeated_inv)
repeated_inv = repeated_inv.explode('liquidation_channel').reset_index(drop = True)

In [81]:
def get_depleted_units_via_liquidation_channel(df):
  if df['priority'] == 'p1':
    if df['liquidation_channel'] == 'friends/family':
      return 0
    elif df['liquidation_channel'] == 'distributor':
      return 0.2 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.2 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.2 * 0.8 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.8 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

  if df['priority'] == 'p2':
    if df['liquidation_channel'] == 'friends/family':
      return 0.6 * 0.05 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'distributor':
      return 0.6 * 0.25 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.6 * 0.20 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.6 * 0.5 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.4 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

  if df['priority'] == 'p3':
    if df['liquidation_channel'] == 'friends/family':
      return 0.8 * 0.1 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'distributor':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'retailer':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'liquidator':
      return 0.8 * 0.3 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'disposal':
      return 0.2 * df['depletion_other_than_pollen']
    elif df['liquidation_channel'] == 'pollen':
      return df['pollen_liquidated_units']

In [82]:
repeated_inv['final_depletion'] = repeated_inv.apply(get_depleted_units_via_liquidation_channel, axis = 1)
repeated_inv['depletion'] = repeated_inv['final_depletion']
repeated_inv = repeated_inv.drop(['pollen_liquidated_units', 'depletion_other_than_pollen', 'final_depletion', 'pollen_recovery_rate_percentage'], axis = 1)

In [84]:
# after above explode, the relationship between time and liquidation channel + time and depletion number + time and shelf_life_remaining_days_prev
# went for a toss. This is for the correction to keep the sensitivity intact

repeated_inv['channel_group'] = repeated_inv.index // len(['friends/family', 'distributor', 'retailer', 'liquidator', 'disposal', 'pollen'])
max_depletion_in_each_group = repeated_inv.groupby('channel_group').agg({'depletion' : 'max'}).reset_index().rename(columns = {'depletion' : 'max_depletion'})
repeated_inv = pd.merge(repeated_inv, max_depletion_in_each_group, 'left', on = 'channel_group')

In [85]:
def time_correction(df):
  if df['max_depletion'] == 0:
    return df['time']
  else:
    return df['time'] * df['depletion'] / df['max_depletion']

repeated_inv['time'] = repeated_inv.apply(time_correction, axis = 1)

def shelf_life_correction(df):
  if df['max_depletion'] == 0:
    return df['shelf_life_remaining_days_prev']
  else:
    return df['shelf_life_remaining_days_prev'] * df['depletion'] / df['max_depletion']

repeated_inv['shelf_life_remaining_days_prev'] = repeated_inv.apply(shelf_life_correction, axis = 1)
repeated_inv = repeated_inv.drop(['channel_group', 'max_depletion'], axis = 1)

In [87]:
sellers['seller_name'] = sellers['seller_name'].str.lower().str.replace(' ', '_')
sellers['persona_seller_type'] = sellers['persona_seller_type'].str.lower().str.replace(' ', '_')
repeated_inv = pd.merge(repeated_inv, sellers, 'left', on = 'seller_name')
print(repeated_inv.priority.isnull().sum(), repeated_inv.persona_seller_type.isnull().sum())

16866 120


In [88]:
repeated_inv['priority'] = np.where(repeated_inv.priority.isnull(), '', repeated_inv.priority)
repeated_inv['persona_seller_type'] = np.where(repeated_inv.persona_seller_type.isnull(), '', repeated_inv.persona_seller_type)

repeated_inv = repeated_inv[~(repeated_inv['total_units_prev'] == 0)]
repeated_inv['depletion_percent'] = repeated_inv['depletion'] / repeated_inv['total_units_prev']
repeated_inv = repeated_inv[~repeated_inv['depletion_percent'].isnull()]
repeated_inv['depletion_percent'] = repeated_inv['depletion_percent']*100

In [89]:
repeated_inv = repeated_inv.rename(columns = {'warehouse_country_prev' : 'warehouse_country', 'shelf_life_remaining_days_prev' : 'shelf_life_remaining_days'})

In [90]:
repeated_inv = repeated_inv.drop_duplicates()
print(repeated_inv.shape)
repeated_inv = repeated_inv.reset_index(drop = True)

(430579, 16)


In [91]:
print(repeated_inv[repeated_inv.time < 0].shape)
repeated_inv['time'] = np.where(repeated_inv.time < 0, -repeated_inv.time, repeated_inv.time)

(720, 16)


In [92]:
priority_to_shelf_life_median = repeated_inv.groupby('priority').shelf_life_remaining_days.median().reset_index()
priority_to_shelf_life_median = dict(zip(list(priority_to_shelf_life_median.priority), list(priority_to_shelf_life_median.shelf_life_remaining_days)))
priority_to_shelf_life_median

{'p1': 78.5704467629027, 'p2': 178.42022214177098, 'p3': 423.3333333333334}

In [93]:
def fill_na_shelf_life_remaining_days(df):
  if pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p1':
    return priority_to_shelf_life_median['p1']
  elif pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p2':
    return priority_to_shelf_life_median['p2']
  elif pd.isna(df['shelf_life_remaining_days']) and df['priority'] == 'p3':
    return priority_to_shelf_life_median['p3']
  else:
    return df['shelf_life_remaining_days']

repeated_inv['shelf_life_remaining_days'] = repeated_inv.apply(fill_na_shelf_life_remaining_days, axis = 1)

In [94]:
repeated_inv.shape

(430579, 16)

In [96]:
train_cols = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'time', 'shelf_life_remaining_days',
              'listing_condition', 'liquidation_channel', 'depletion_percent']
cat_cols = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'listing_condition', 'liquidation_channel']

print(repeated_inv[train_cols].duplicated().sum())
repeated_inv = repeated_inv.drop_duplicates(subset = train_cols)

85422


In [98]:
X_train, X_val, y_train, y_val = train_test_split(repeated_inv[train_cols], repeated_inv['possible'], test_size = 0.2, random_state = 7, stratify = repeated_inv['possible'])

In [99]:
model_params = {'loss_function': 'Logloss',
          'od_wait' : 25,
          'learning_rate': 0.6,
          'depth': 7,
          'min_data_in_leaf': 10}

In [100]:
def train_catboost(X_train, y_train, X_val, y_val, cat_features, params, iterations = 300, eval_metric = 'Logloss', random_state = 7):
  model = CatBoostClassifier(**model_params, iterations = iterations, eval_metric = eval_metric, random_state = random_state)
  model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_val, y_val), verbose=True)
  return model

In [101]:
# auto_class_weights = 'Balanced'

In [102]:
sales_probability_model = train_catboost(X_train, y_train, X_val, y_val, cat_cols, model_params, iterations = 1000, eval_metric = 'Logloss', random_state = 7)

0:	learn: 0.6024680	test: 0.6020846	best: 0.6020846 (0)	total: 671ms	remaining: 11m 10s
1:	learn: 0.5881441	test: 0.5881477	best: 0.5881477 (1)	total: 1.35s	remaining: 11m 16s
2:	learn: 0.5794461	test: 0.5793897	best: 0.5793897 (2)	total: 2.06s	remaining: 11m 23s
3:	learn: 0.5580841	test: 0.5503537	best: 0.5503537 (3)	total: 2.61s	remaining: 10m 49s
4:	learn: 0.5402742	test: 0.5109055	best: 0.5109055 (4)	total: 3.14s	remaining: 10m 24s
5:	learn: 0.5257926	test: 0.4803743	best: 0.4803743 (5)	total: 3.87s	remaining: 10m 40s
6:	learn: 0.5233103	test: 0.4779871	best: 0.4779871 (6)	total: 4.53s	remaining: 10m 42s
7:	learn: 0.5207418	test: 0.4705542	best: 0.4705542 (7)	total: 5.04s	remaining: 10m 24s
8:	learn: 0.5175087	test: 0.4675375	best: 0.4675375 (8)	total: 5.65s	remaining: 10m 22s
9:	learn: 0.5137990	test: 0.4574569	best: 0.4574569 (9)	total: 6.29s	remaining: 10m 22s
10:	learn: 0.5102210	test: 0.4545022	best: 0.4545022 (10)	total: 6.95s	remaining: 10m 25s
11:	learn: 0.5083904	test: 0.4

In [104]:
y_val[:30].values

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0])

In [105]:
pred_probs = sales_probability_model.predict_proba(X_val)
pred_probs[:30][:,1]

array([4.39198204e-02, 2.60058597e-03, 2.09973356e-02, 1.54301690e-01,
       9.25496633e-01, 8.94786619e-01, 4.16982372e-01, 4.75258277e-01,
       1.68423247e-05, 1.79562553e-01, 4.40086527e-02, 9.63001987e-01,
       2.07169103e-01, 3.24395306e-01, 6.82226195e-03, 9.36157066e-01,
       7.68144042e-02, 7.32619651e-01, 2.01662361e-02, 6.78307609e-01,
       5.47477339e-02, 1.88452692e-01, 4.11208671e-02, 1.74819614e-02,
       3.84262540e-02, 8.89958679e-01, 9.98163621e-01, 5.33848426e-04,
       9.77461056e-01, 2.86668289e-01])

In [107]:
# # tune the threshold

# def plot_mterics_report(y_val, pred_probs, t):
#   bc = BinaryClassification(y_val, pred_probs[:, 1], labels=["Not Sellable", "Sellable"])

#   # Figures
#   plt.figure(figsize=(15,10))
#   plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
#   bc.plot_roc_curve(threshold = t)
#   plt.subplot2grid((2,6), (0,2), colspan=2)
#   bc.plot_precision_recall_curve(threshold = t)
#   plt.subplot2grid((2,6), (0,4), colspan=2)
#   bc.plot_class_distribution(pal_colors=['r','g','m','k'], threshold = t)
#   plt.subplot2grid((2,6), (1,1), colspan=2)
#   a = bc.plot_confusion_matrix(threshold = t)

#   plt.show()
#   bc.print_report(threshold = t)

# plot_mterics_report(y_val, pred_probs, t = 0.4)

In [ ]:
# ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'time', 'shelf_life_remaining_days',
#               'listing_condition', 'liquidation_channel', 'depletion_percent']

In [185]:
repeated_inv[repeated_inv.possible == 1].sample()[train_cols].values[0]

array(['G4246600', "l'oreal_paris", 'hair_care', 'hair_color',
       "l'oreal_thailand", 27.0, 535.0, 'excess', 'retailer',
       21.79591836734694], dtype=object)

In [198]:
test_data_point = ['G4246600', "l'oreal_paris", 'hair_care', 'hair_color',
       "l'oreal_thailand", 60.0, 500.0, 'excess', 'retailer',
       70]

In [199]:
sales_probability_model.predict_proba(test_data_point)[1]

0.061740172250280896

In [122]:
# def dump_files_for_inference_pipeline():
#   print('Dumping files for inference later ...')
#   joblib.dump(sales_probability_model, '/content/drive/MyDrive/PSI_Modeling_Data/depletion_model/sales_probability_model.pkl')

In [123]:
# dump_files_for_inference_pipeline()